<a href="https://colab.research.google.com/github/shubshub-bol/FlightFarePred/blob/main/Flight_Price_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a id = '5.5'></a>
<p style = "font-size : 25px; color : #34656d ; font-family : 'Comic Sans MS'; text-align : center; background-color : #fbc6a4; border-radius: 5px 5px;"><strong>Flight Price Prediction</strong></p>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import accuracy_score,confusion_matrix

import warnings
warnings.filterwarnings('ignore')


# Importing dataset
1.Since data is in form of excel file we have to use pandas read_excel to load the data.   
2.After loading it is important to check null values in a column or a row           
3.If it is present then following can be done,                                                                                   
*       Filling NaN values with mean, median and mode using fillna() method                                                           
*       If Less missing values, we can drop it as well           

In [ ]:
#importing data
df=pd.read_excel('/content/Flight_Fare.xlsx')
df.head()


In [ ]:
 # creating it for future use case
 df2=df.copy()
 df3=df.copy()

In [ ]:
#information about the data
df.info()


In [ ]:
#describe about the data
df.describe()

In [ ]:
df.shape

In [ ]:
#finding the null values
df.isnull().sum()

### **We observe 2 missing values,So directly drop these as these are very less.**

In [ ]:
#drop the nullvalues
df.dropna(inplace=True)

In [ ]:
df.isnull().sum()

<a id = '5.5'></a>
<p style = "font-size : 25px; color : 'blue' ; font-family : 'Comic Sans MS'; text-align : center; background-color : #fbc6a4; border-radius: 5px 5px;"><strong>Data Cleaning</strong></p>

In [ ]:
# datatypes
df.dtypes

#### The datatypes of Date_of_journey,Arrival_Time and Dep_Time is object.So,firstly we convert it into date and time for proper predicion.

**dt.day method will extract only day of that date                           
dt.month method will extract only month of that date**

In [ ]:
def change_into_datetime(col):
    df[col]=pd.to_datetime(df[col])

In [ ]:
df.columns

In [ ]:
for i in ['Date_of_Journey','Dep_Time', 'Arrival_Time']:
    change_into_datetime(i)

In [ ]:
df.dtypes

##### Now ,we extract day and month from Date_of_journey and stored in 2 other columns.
##### Then there will be no use of 'Date_of_Journey' column,so we drop it.

In [ ]:
df['journey_day']=df['Date_of_Journey'].dt.day
df['journey_month']=df['Date_of_Journey'].dt.month


In [ ]:
df.head(10)

In [ ]:
df.drop('Date_of_Journey', axis=1, inplace=True)

##### From Arrival_time and Dept_time features,we extract hour and minutes and stored in new columns and drop these columns

In [ ]:
# function for extracting hour and minutes
def extract_hour(data,col):
    data[col+'_hour']=data[col].dt.hour

def extract_min(data,col):
    data[col+'_min']=data[col].dt.minute


def drop_col(data,col):
    data.drop(col,axis=1,inplace=True)

In [ ]:
#call the function
# Departure time is when a plane leaves the gate.
# Similar to Date_of_Journey we can extract values from Dep_Time
extract_hour(df,'Dep_Time')

#extracting minutes
extract_min(df,'Dep_Time')

#drop the column
drop_col(df,'Dep_Time')

In [ ]:
#extracting hour
extract_hour(df,'Arrival_Time')

#extracting min
extract_min(df,'Arrival_Time')


#drop the column
drop_col(df,'Arrival_Time')

In [ ]:
df.head(10)

##### Lets Apply pre-processing on duration column,Separate Duration hours and minute from duration

In [ ]:
duration=list(df['Duration'])
for i in range(len(duration)):
    if len(duration[i].split(' '))==2:
        pass
    else:
        if 'h' in duration[i]: # Check if duration contains only hour
             duration[i]=duration[i] + ' 0m' # Adds 0 minute
        else:
             duration[i]='0h '+ duration[i]

In [ ]:
df['Duration']=duration

In [ ]:
df.head()

In [ ]:
def hour(x):
    return x.split(' ')[0][0:-1]

def minutes(x):
    return x.split(' ')[1][0:-1]



In [ ]:
df['dur_hour']=df['Duration'].apply(hour)

In [ ]:
df['dur_min']=df['Duration'].apply(minutes)

In [ ]:
df.head(10)

In [ ]:
drop_col(df,'Duration')

In [ ]:
df.dtypes

In [ ]:
df['dur_hour'] = df['dur_hour'].astype(int)
df['dur_min'] = df['dur_min'].astype(int)

In [ ]:
df.dtypes

#### Finding the categorical value

## Airline vs Price Analysis

In [ ]:
plt.figure(figsize=(15, 8))
sns.boxplot(
    x='Airline',
    y='Price',
    data=df.sort_values('Price', ascending=False),
    palette='Set2'  # Try 'Set2', 'husl', 'coolwarm', 'Spectral', etc.
)

plt.title("Airline vs Price Distribution", fontsize=16, fontweight='bold')
plt.xticks(rotation=45)
plt.ylabel("Price (INR)")
plt.xlabel("Airline")
plt.tight_layout()
plt.show()



### From graph we can see that Jet Airways Business have the highest Price., Apart from the first Airline almost all are having similar median

#### Perform Total_Stops vs Price Analysis

In [ ]:
plt.figure(figsize=(15,8))
sns.boxplot(x='Total_Stops',y='Price',data=df.sort_values('Price',ascending=False),palette='husl' )
plt.title("Total_Stops vs Price Distribution", fontsize=16, fontweight='bold')
plt.xticks(rotation=45)
plt.ylabel("Price (INR)")
plt.xlabel("Total_Stops")
plt.tight_layout()
plt.show()

# Handling categorical data




### We are using two main Encoding Techniques to covert Categorical data into some numerical format

#### Nominal data -- Data that are not in any order -->one hot encoding
#### ordinal data -- Data are in order --> labelEncoder

In [ ]:
# As Airline is Nominal Categorical data we will perform OneHotEncoding
Airline=pd.get_dummies(df['Airline'],drop_first=True)

In [ ]:
Airline.head()

In [ ]:
# Value counts for Source and Destination
source_counts = df2['Source'].value_counts()
destination_counts = df2['Destination'].value_counts()

# Create subplots
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Style settings
sns.set_style("whitegrid")
palette_src = sns.color_palette("viridis", len(source_counts))
palette_dest = sns.color_palette("plasma", len(destination_counts))

# Source plot
sns.barplot(x=source_counts.index, y=source_counts.values, palette=palette_src, ax=axes[0])
axes[0].set_title('Flights by Source City', fontsize=16, fontweight='bold')
axes[0].set_xlabel('Source City', fontsize=12)
axes[0].set_ylabel('Number of Flights', fontsize=12)
axes[0].tick_params(axis='x', rotation=45)

# Destination plot
sns.barplot(x=destination_counts.index, y=destination_counts.values, palette=palette_dest, ax=axes[1])
axes[1].set_title('Flights by Destination City', fontsize=16, fontweight='bold')
axes[1].set_xlabel('Destination City', fontsize=12)
axes[1].set_ylabel('Number of Flights', fontsize=12)
axes[1].tick_params(axis='x', rotation=45)

# Layout
plt.tight_layout()
plt.show()


In [ ]:
#encoding of source column
source=pd.get_dummies(df['Source'],drop_first=True)
source.head()

In [ ]:
plt.figure(figsize=(15,8))
sns.boxplot(x='Destination',y='Price',data=df.sort_values('Price',ascending=False),palette='husl')
plt.title("Source vs Price Distribution", fontsize=16, fontweight='bold')
plt.ylabel("Price (INR)")
plt.xlabel("Source")
plt.tight_layout()
plt.show()

In [ ]:
#encoding of destination column
destination=pd.get_dummies(df['Destination'],drop_first=True)
destination.head()

In [ ]:
# now work on route column
df['Route'].value_counts()


In [ ]:
df['Route1'] = df['Route'].str.split('→').str[0]
df['Route2'] = df['Route'].str.split('→').str[1]
df['Route3'] = df['Route'].str.split('→').str[2]
df['Route4'] = df['Route'].str.split('→').str[3]
df['Route5'] = df['Route'].str.split('→').str[4]


In [ ]:
df.head()

In [ ]:
drop_col(df,'Route')

In [ ]:
df

In [ ]:
df.isnull().sum()

In [ ]:
df.columns

In [ ]:
for i in ['Route3', 'Route4', 'Route5']:
    df[i].fillna('None',inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
for i in df.columns:
    print('{} has total {} categories'.format(i,len(df[i].value_counts())))

In [ ]:
# Applying label encoder
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [ ]:
for i in ['Route1', 'Route2', 'Route3', 'Route4', 'Route5']:
    df[i]=encoder.fit_transform(df[i])

In [ ]:
df.head()

In [ ]:
drop_col(df,'Additional_Info')

In [ ]:
df['Total_Stops'].unique()

In [ ]:
# encoding Total stops
dict={'non-stop':0, '2 stops':2, '1 stop':1, '3 stops':3, '4 stops':4}
df['Total_Stops']=df['Total_Stops'].map(dict)

In [ ]:
df['Total_Stops']


In [ ]:
drop_col(df,'Source')
drop_col(df,'Destination')
drop_col(df,'Airline')

<a id = '5.5'></a>
<p style = "font-size : 25px; color : #34656d ; font-family : 'Comic Sans MS'; text-align : center; background-color : #fbc6a4; border-radius: 5px 5px;"><strong>After all preprocessing, Our data is ready for the modeling</strong></p>

In [ ]:
final_df=pd.concat([df,Airline,source,destination],axis=1)

In [ ]:
final_df.head()

In [ ]:
pd.set_option('display.max_columns',33)
final_df.head()

## Check For Outliers


In [ ]:
def plot(data, col):
    # Set style
    sns.set_style("whitegrid")

    # Create subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8), gridspec_kw={'height_ratios': [3, 1]})

    # Distribution plot
    sns.histplot(data[col], kde=True, color='skyblue', ax=ax1)
    ax1.set_title(f"Distribution of {col}", fontsize=16, fontweight='bold')
    ax1.set_xlabel("")
    ax1.set_ylabel("Frequency")

    # Box plot
    sns.boxplot(x=data[col], ax=ax2, palette="Set2")
    ax2.set_title(f"Boxplot of {col}", fontsize=16, fontweight='bold')
    ax2.set_xlabel(col)

    # Adjust layout
    plt.tight_layout()
    plt.show()


In [ ]:
plot(final_df,'Price')

###  Handling outliers:
#### As there is some outliers in price feature,so we replace it  with median.

In [ ]:
final_df['Price']=np.where(final_df['Price']>=40000,final_df['Price'].median(),final_df['Price'])

In [ ]:
plot(final_df,'Price')

### Seprate the dataset in X and Y columns

In [ ]:
X=final_df.drop('Price',axis=1)
y=df['Price']

# Feature Selection
It is about finding the best feature and having good relation with independent variable                                  
Issue of dimensionality reduction reduces.

For this we use mutual_info_classif

In [ ]:
from sklearn.feature_selection import mutual_info_classif


In [ ]:
mutual_info_classif(X,y)

In [ ]:
imp = pd.DataFrame(mutual_info_classif(X,y),index=X.columns)
imp

In [ ]:
imp.columns=['importance']
imp.sort_values(by='importance',ascending=False)

As we can see from the table, there are several features that have approx 0 value,so we should remove it after authorizing it.But for now,we are not removing it.

In [ ]:
# Calculate MI scores
mi_scores = mutual_info_classif(X, y, discrete_features='auto')

# Create DataFrame
mi_df = pd.DataFrame({'Feature': X.columns, 'MI Score': mi_scores})
mi_df = mi_df.sort_values(by='MI Score', ascending=False)

# Plot
plt.figure(figsize=(10,6))
sns.barplot(x='MI Score', y='Feature', data=mi_df, palette='viridis')
plt.title('Feature Importance (Mutual Information)', fontsize=16)
plt.xlabel('Mutual Information Score')
plt.ylabel('Feature')
plt.show()

 * Top Features

Route2, Route3, and Route1 dominate the list.
→ This means the middle legs of the journey carry the most predictive power for ticket price. Probably because connecting airports affect cost significantly.

Total_Stops also ranks very high, which makes sense since more stops generally mean cheaper fares (low-cost carriers) or sometimes higher (due to long-haul complexity).

* Timing Features

Arrival_Time_hour and Dep_Time_hour have strong importance.
→ Certain times of day are likely in higher demand (e.g., early morning or evening peak flights).

Duration-related features (dur_hour, dur_min) also matter, reflecting the relationship between travel time and cost.

* Date-based Features

journey_month ranks higher than journey_day, meaning month-based seasonality influences fares more than specific days of the month.
(e.g., holiday seasons, summer breaks, etc.)

* Low-Importance Features

Specific airlines like GoAir, Trujet, and premium-class indicators have low mutual information.
→ Could be because their presence in the dataset is rare or because airline brand is less predictive than route/timing.



In [ ]:
# Helper: split route into list of airports
def split_airports(route):
    if pd.isna(route):  # Skip if NaN
        return []
    return [x.strip() for x in str(route).split('→')]

# Count take-offs and landings
take_off_counts = {}
landing_counts = {}

for route in df3['Route']:
    airports = split_airports(route)
    if not airports:  # Skip empty
        continue

    # First airport = take-off
    take_off_counts[airports[0]] = take_off_counts.get(airports[0], 0) + 1

    # Last airport = landing
    landing_counts[airports[-1]] = landing_counts.get(airports[-1], 0) + 1

    # Intermediate hops: each stop has a landing then a take-off
    for i in range(1, len(airports) - 1):
        landing_counts[airports[i]] = landing_counts.get(airports[i], 0) + 1
        take_off_counts[airports[i]] = take_off_counts.get(airports[i], 0) + 1

# Merge into one DataFrame
airports_df = pd.DataFrame({
    'Take-offs': pd.Series(take_off_counts),
    'Landings': pd.Series(landing_counts)
}).fillna(0)

airports_df['Total Movements'] = airports_df['Take-offs'] + airports_df['Landings']
airports_df = airports_df.sort_values('Total Movements', ascending=False)

# Plot
airports_df.head(15)[['Take-offs', 'Landings']].plot(
    kind='barh', stacked=True, color=['#1f77b4', '#ff7f0e'], figsize=(12, 8)
)

plt.title('Busiest Airports by Take-offs and Landings (Including Stops)', fontsize=16, weight='bold')
plt.xlabel('Number of Movements')
plt.ylabel('Airport Code')
plt.gca().invert_yaxis()
plt.grid(axis='x', linestyle='--', alpha=0.6)
plt.legend(title='Movement Type')
plt.tight_layout()
plt.show()


| IATA Code | Full Airport Name                                                                         |
| --------- | ----------------------------------------------------------------------------------------- |
| **BOM**   | Chhatrapati Shivaji Maharaj International Airport — Mumbai         |
| **DEL**   | Indira Gandhi International Airport — Delhi
| **BLR**   | Kempegowda International Airport — Bengaluru
| **COK**   | Cochin International Airport — Kochi
| **CCU**   | Netaji Subhas Chandra Bose International Airport — Kolkata
| **HYD**   | Rajiv Gandhi International Airport — Hyderabad\
| **MAA**   | Chennai International Airport — Chennai
| **AMD**   | Sardar Vallabhbhai Patel International Airport — Ahmedabad
| **JAI**   | Jaipur International Airport — Jaipur
| **BBI**   | Biju Patnaik International Airport — Bhubaneswar
| **NAG**   | Dr. Babasaheb Ambedkar International Airport — Nagpur
| **GAU**   | Lokpriya Gopinath Bordoloi International Airport — Guwahati
| **GOI**   | Goa International Airport (Dabolim) — Goa
| **PNQ**   | Pune Airport — Pune           |
| **IDR**   | Devi Ahilya Bai Holkar Airport — Indore



In [ ]:
# Create a journey column
df2['Journey'] = df2['Source'] + " → " + df2['Destination']

# Group by journey for count and average price
journey_stats = df2.groupby('Journey').agg(
    Frequency=('Journey', 'size'),
    Avg_Price=('Price', 'mean')
).reset_index()

# Sort by frequency
journey_stats = journey_stats.sort_values(by='Frequency', ascending=False).head(10)

# Plot
fig, ax1 = plt.subplots(figsize=(16,6))

# Bar plot for frequency
sns.barplot(data=journey_stats, x='Journey', y='Frequency', palette='crest', ax=ax1)
ax1.set_ylabel('Frequency', color='b')
ax1.set_xlabel('Journey')
ax1.tick_params(axis='y', labelcolor='b')
ax1.set_xticklabels(ax1.get_xticklabels(), rotation=45, ha='right')

# Secondary axis for average price
ax2 = ax1.twinx()
sns.lineplot(data=journey_stats, x='Journey', y='Avg_Price', marker='o', color='r', ax=ax2)
ax2.set_ylabel('Average Price (₹)', color='r')
ax2.tick_params(axis='y', labelcolor='r')

plt.title("Most Frequent Journeys & Their Average Price")
plt.tight_layout()
plt.show()


In [ ]:

from collections import Counter


# Remove NaN and split routes
all_stops = df2['Route'].dropna().apply(lambda x: x.split(' → '))

# Count intermediate stops
connecting_airports = Counter()
for route in all_stops:
    if len(route) > 2:  # has at least 1 connecting airport
        connecting_airports.update(route[1:-1])

# Create dataframe of top 10
top_connecting = connecting_airports.most_common(10)
connecting_df = pd.DataFrame(top_connecting, columns=['Airport', 'Count'])

# Plot
plt.figure(figsize=(8,5))
sns.barplot(data=connecting_df, x='Count', y='Airport', palette='viridis')
plt.title("Top 10 Most Used Connecting Airports(layovers)")
plt.xlabel("Airport Code")
plt.ylabel("Count")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()



**Insight & Opportunity:**  

Data shows **BOM (Mumbai)** is by far the most used connecting airport, followed by DEL, HYD, and others.  
Because of layovers, these airports have more advantage in restaurant-related businesses, cafés, and other F&B outlets, as passengers often have idle time.

**Additional Revenue Stream:**

Installing **self-service vending machines** at these top layover hubs—especially BOM—can tap into heavy foot traffic from transfer passengers.

**Product Mix:**  
- **Travel essentials:** chargers, earphones, neck pillows, SIM cards  
- **Snacks & drinks:** bottled water, coffee cans, energy bars  
- **Local souvenirs:** packaged sweets, small handicrafts  

**Why it works:**  
High layover volumes + limited time between flights = strong demand for quick, convenient purchases and food options.  
Low staffing needs make vending scalable across airports.

In [ ]:
# Example stop counts (replace these with your actual data counts)
stop_counts = [
    (df2['Total_Stops'] == '1 stop').sum(),          # One Stop
    (df2['Total_Stops'] == 'non-stop').sum(),        # Zero Stops
    (df2['Total_Stops'].isin(['2 stops', '3 stops'])).sum()  # Two Or More
]

labels = ['One Stop', 'Non Stop', 'Two Or More']
colors = ['#4e79a7', '#59a14f', '#e15759']

# Donut chart
plt.figure(figsize=(6,6))
wedges, texts, autotexts = plt.pie(
    stop_counts,
    labels=labels,
    autopct='%1.1f%%',
    colors=colors,
    wedgeprops={'width': 0.4}
)

# Add center circle
plt.gca().add_artist(plt.Circle((0, 0), 0.70, fc='white'))

plt.title("stops", fontsize=16)
plt.show()

# Models

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor

In [ ]:
# spiliting the dataset
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,random_state=123)

In [ ]:
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
def predict(ml_model):
    print('Model is: {}'.format(ml_model))
    model= ml_model.fit(X_train,y_train)
    print("Training score: {}".format(model.score(X_train,y_train)))
    predictions = model.predict(X_test)
    print("Predictions are: {}".format(predictions))
    print('\n')
    r2score=r2_score(y_test,predictions)
    print("r2 score is: {}".format(r2score))

    print('MAE:{}'.format(mean_absolute_error(y_test,predictions)))
    print('MSE:{}'.format(mean_squared_error(y_test,predictions)))
    print('RMSE:{}'.format(np.sqrt(mean_squared_error(y_test,predictions))))

    sns.distplot(y_test-predictions)


In [ ]:

predict(RandomForestRegressor())

In [ ]:
predict(LinearRegression())



In [ ]:
predict(KNeighborsRegressor())


In [ ]:
predict(DecisionTreeRegressor())

In [ ]:
from sklearn.svm import SVR
predict(SVR())

In [ ]:
predict(GradientBoostingRegressor())

# Hypertunning the model

In [ ]:
from sklearn.model_selection import RandomizedSearchCV


In [ ]:
random_grid = {
    'n_estimators' : [100, 120, 150, 180, 200,220],
    'max_features':['auto','sqrt'],
    'max_depth':[5,10,15,20],
    }

In [ ]:
rf=RandomForestRegressor()
rf_random=RandomizedSearchCV(estimator=rf,param_distributions=random_grid,cv=3,verbose=2,n_jobs=-1,)

rf_random.fit(X_train,y_train)

# best parameter
rf_random.best_params_

In [ ]:
# best parameter
rf_random.best_params_

In [ ]:
#predicting the values
prediction = rf_random.predict(X_test)

#distribution plot between actual value and predicted value
sns.displot(y_test-prediction)

In [ ]:
r2_score(y_test,prediction)

After hypertuning accuracy incrased .

# ✈️ Flight Price Prediction – Project Report

**Objective**  
Predict flight ticket prices using machine learning to help customers and airlines in decision-making.

---

### 📊 Dataset  
- Features: Airline, Source, Destination, Total_Stops, Duration, Route, Additional_Info  
- Target: Price  

---

### ⚙️ Preprocessing  
- Converted Duration → minutes  
- Extracted Departure/Arrival hours  
- Encoded categorical features (Airline, Source, Destination, Stops)  
- Scaled numerical values  

---

### 🔍 EDA Highlights  
- Non-stop flights are costliest  
- Airline strongly affects price  
- Duration and Stops are major predictors  

---

### 🤖 Models Tried  
- Linear Regression (baseline)  
- Decision Tree, KNN  
- Random Forest ✅ (best)  
- Gradient Boosting (good but slower)  

---

### 📈 Results  
- **Random Forest Regressor** → Best accuracy (highest R², lowest RMSE)  
- Gradient Boosting close second  

---

### 📝 Insights  
- Fewer stops → higher prices  
- Some airlines consistently more expensive  
- Flight duration correlates with price  

---

### 🚀 Future Work  
- Try XGBoost / LightGBM  
- Build Flask/Streamlit app  
- Add real-time dynamic pricing  

---

**Tools Used**: Python, Pandas, NumPy, Matplotlib, Seaborn, scikit-learn  
